In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
import glob
sns.set_style('white')

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "/Users/sbandya/miniconda3/envs/napari-env/bin/python"
  * The NumPy version is: "1.22.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/sbandya/miniconda3/envs/napari-env/lib/python3.9/site-packages/numpy/core/_multiarray_umath.cpython-39-darwin.so, 0x0002): Library not loaded: '@rpath/libcblas.3.dylib'
  Referenced from: '/Users/sbandya/miniconda3/envs/napari-env/lib/python3.9/site-package

In [2]:
!pip3 install pandas

In [3]:
sample_list = pd.read_csv("/Users/sbandya/Desktop/dev/hemocyte_ImageSegmentation/sample_list.csv",sep=",",header=0)

am_path= "/Users/sbandya/Desktop/dev/hemocyte_ImageSegmentation/analysis/annotation_measurements/"
dm_path= "/Users/sbandya/Desktop/dev/hemocyte_ImageSegmentation/analysis/detection_measurements/"

figure_outputs = "/Users/sbandya/Desktop/dev/hemocyte_ImageSegmentation/analysis/figures/"
sample_list[:5]


,Sample #,Genotype,Condition,n,date
0,1,HML-GFP,C,8,9/27/2020
1,2,HML-GFP,C,7,9/27/2020
2,3,roGFP,C,10,9/27/2020
3,4,roGFP,C,9,9/27/2020
4,5,drpr∆5,C,7,9/27/2020


In [4]:
## Create a dataframe of annotation measurements for all images

def get_sample_condition(imagename,sample_mapping):
    sampleid = imagename[:2]
    condition = sample_mapping[sample_mapping['Sample #'] == int(sampleid)]
    return condition['Condition'].iloc[0]

all_annotation_measurement_files = glob.glob(path.join(am_path , "*.txt"))

am_list = []
for amf in all_annotation_measurement_files:
    df = pd.read_csv(amf, index_col=None, header=0, sep="\t")
    am_list.append(df)

annotation_measurements = pd.concat(am_list, axis=0, ignore_index=True)
annotation_measurements['condition'] = annotation_measurements['Image'].apply(get_sample_condition,sample_mapping=sample_list)
#annotation_measurements = annotation_measurements.loc[annotation_measurements["condition"] == "C", "condition"] = 'Fed'
annotation_measurements['condition'] = np.where(annotation_measurements['condition'] == 'C', 'Fed', annotation_measurements['condition'])
annotation_measurements['condition'] = np.where(annotation_measurements['condition'] == 'S', 'Starved', annotation_measurements['condition'])

annotation_measurements.columns = annotation_measurements.columns.str.replace('Centroid X µm', 'Centroid_X')
annotation_measurements.columns = annotation_measurements.columns.str.replace('Centroid Y µm', 'Centroid_Y')
print(annotation_measurements.shape)
annotation_measurements[:5]



(275, 14)


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,condition
0,73F_3_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,428.06,311.66,41,41,100,303.92,134903.3,1435.90,Fed
1,73F_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,429.21,323.35,39,39,100,251.08,155328.9,1670.40,Fed
2,60C_5_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,450.16,281.02,10,10,100,53.41,187221.5,1667.40,Fed
3,52E_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,210.09,164.56,10,10,100,233.14,42891.9,797.06,Fed
4,60C_4_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,454.92,327.59,27,27,100,153.15,176300.1,1744.70,Fed


In [60]:
from scipy.spatial import distance
import math 
sns.set_palette("pastel")

def get_centroid_distance(image,x,y,polygon):
    polygon_x = annotation_measurements[annotation_measurements['Image']==image]['Centroid_X'].iloc[0]
    polygon_y = annotation_measurements[annotation_measurements['Image']==image]['Centroid_Y'].iloc[0]
    centroid_distance = distance.euclidean([polygon_x, polygon_y], [x,y])

    return centroid_distance
    

all_detection_measurement_files = glob.glob(path.join(dm_path , "*.txt"))
centroid_distances = {}
fed_centroid_distances = list()
starved_centroid_distances = []

for dm in all_detection_measurement_files:
    detection_measurements = pd.read_csv(dm,sep="\t",header=0)
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid X µm', 'Centroid_X')
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid Y µm', 'Centroid_Y')
    

        
    detection_measurements["centroid_distances"] = detection_measurements.apply(
        lambda x: get_centroid_distance(x.Image, x.Centroid_X,x.Centroid_Y,polygon=annotation_measurements), axis=1)
    
    #print(detection_measurements[:5])
    
    centroid_distances[detection_measurements['Image'].iloc[0]] =  detection_measurements['centroid_distances'].to_list()
    #print(centroid_distances)
    
    condition = annotation_measurements[annotation_measurements['Image'] == detection_measurements['Image'].iloc[0]]
    
    #print(condition['condition'].iloc[0])
    
    if condition['condition'].iloc[0] == 'Fed':
        fed_centroid_distances.append(detection_measurements['centroid_distances'].to_list())
        #sns.histplot(detection_measurements['centroid_distances'].to_list(),kde=False, color='xkcd:azure', bins=3)
    else: 
        starved_centroid_distances.append(detection_measurements['centroid_distances'].to_list())
        #sns.histplot(detection_measurements['centroid_distances'].to_list(),kde=False, color='orange',bins=3)
    


In [69]:
for key in centroid_distances.keys():
    if "60C" in key:
        print(centroid_distances[key])

[249.02114870026602, 162.65965848974352, 233.30829218011084, 189.39833262201654, 169.62979484748547, 261.9162707813319, 266.99019345286825, 170.1152776795782, 237.26351468356864, 200.2112956353862]
[180.8368823000441, 176.6273738693977, 284.1919759599134, 260.49024281151105, 262.0296166466684, 275.2426676589224, 8.055966732801235, 18.648029386506206, 170.8308359167045, 173.80376060373376, 272.7955146258824, 280.3063980718243, 265.03039410603463, 265.3355168461245, 74.08714125946558, 95.49803191689348, 289.74147252335143, 251.89267099302432, 307.6803861152024, 229.56507857250418, 132.58630095149348, 230.45954894514568, 223.25573340902133, 136.116685237336, 182.89409093789774, 141.92490725732395, 171.80061059262863]
[270.53700024211105, 223.65031030606684, 282.4042954701645, 229.5737626559272, 119.66034681547602, 228.42365661200682, 223.5975608095938, 112.78297433566827, 119.4302411451974, 128.39701709930807, 252.20950220005594]
[123.18097458617548, 199.91125080895273, 254.86819534810542

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')


ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
plt.show()